In [13]:
!conda install -c conda-forge opencv pyopengl

Channels:
 - conda-forge
 - defaults
Platform: win-64



CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://conda.anaconda.org/conda-forge/win-64/repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.
'https//conda.anaconda.org/conda-forge/win-64'




In [8]:
import cv2
import mediapipe as mp
import numpy as np
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import tensorflow as tf

ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
# Initialize MediaPipe Pose for pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
# Initialize motion detection using OpenCV's background subtraction
fgbg = cv2.createBackgroundSubtractorMOG2()

In [ ]:
# Initialize OpenCV VideoCapture
video_path = 'path_to_video.mp4'
cap = cv2.VideoCapture(video_path)

In [ ]:
# OpenGL window setup
def setup():
    glClearColor(0.0, 0.0, 0.0, 1.0)  # Black background
    glEnable(GL_DEPTH_TEST)

def draw_3d_objects(landmarks):
    """
    Use OpenGL to draw the pose landmarks in 3D space.
    """
    glBegin(GL_POINTS)
    for landmark in landmarks:
        x, y, z = landmark
        glVertex3f(x, y, z)
    glEnd()


In [ ]:
def render_frame(frame, landmarks):
    """
    This function will render the 3D objects on the video frame
    using OpenGL.
    """
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()

    # Set the camera
    glTranslatef(0.0, 0.0, -5)  # Move back to see objects

    # Draw the 3D pose landmarks
    draw_3d_objects(landmarks)
    
    # Render the video frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.array(frame, dtype=np.uint8)
    glDrawPixels(frame.shape[1], frame.shape[0], GL_RGB, GL_UNSIGNED_BYTE, frame)

    glutSwapBuffers()

In [ ]:
# Process video frames and detect motion/pose
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Motion detection: Get moving objects
    fgmask = fgbg.apply(frame)
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 2. Pose detection with MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    # 3. Get 3D landmarks for pose (assumed to be in world coordinates, simple conversion)
    landmarks_3d = []
    if result.pose_landmarks:
        for landmark in result.pose_landmarks.landmark:
            landmarks_3d.append([landmark.x * 10, landmark.y * 10, landmark.z * 10])  # Scale for visibility

    # 4. Setup OpenGL rendering
    glutInit()
    glutInitDisplayMode(GLUT_RGB | GLUT_DOUBLE | GLUT_DEPTH)
    glutInitWindowSize(frame.shape[1], frame.shape[0])
    glutCreateWindow(b"Cricket 3D Rendering")

    setup()

    # Render the 3D scene (pose landmarks)
    render_frame(frame, landmarks_3d)

    # 5. Display the frame in a separate window
    cv2.imshow('Motion Detected', fgmask)

    # Close on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
# Cleanup
cap.release()
cv2.destroyAllWindows()